In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# ============================
# 1. Selecciona el dataset
# ============================

df_flattened_list = pd.read_csv('clips_features_flattened_list.csv')
df_flattened_list_pca = pd.read_csv('clips_features_flattened_list_pca.csv')

df_flattened = pd.read_csv('clips_features_flattened.csv')
df_flattened_pca = pd.read_csv('clips_features_flattened_pca.csv')

df = df_flattened_pca.copy()   # <<-- cambia por el dataset que quieras probar


FileNotFoundError: [Errno 2] No such file or directory: 'clips_features_flattened_list.csv'

: 

In [ ]:
# ============================
# 2. Features (X) y Labels (y)
# ============================
y = df["class_id"].values
X = df.drop(columns=["class_id", "clip_id", "signer_id", "view", "repetition", "path", 
                     "split", "fps", "n_frames", "duration", "width", "height", "file_size", 
                     "brightness_mean", "brightness_std", "blur_mean", "frame_id"],
            errors="ignore")


In [ ]:
# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Codificación de labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [ ]:
# Split en train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_enc, test_size=0.2, random_state=42, stratify=y_enc)


In [ ]:
# ============================
# 3. Modelos Clásicos
# ============================
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(),
    "MLP": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300)
}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print(f"\n=== {nombre} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
# ============================
# 4. RNN con LSTM / GRU
# ============================
# Ajustar dimensiones para RNN: (samples, timesteps, features)
# Aquí tratamos cada fila como una secuencia de 1 timestep
X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)

# LSTM
model_lstm = models.Sequential([
    layers.LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    layers.Dense(len(le.classes_), activation="softmax")
])

model_lstm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_lstm.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

loss, acc = model_lstm.evaluate(X_test_rnn, y_test)
print("\n=== LSTM ===")
print("Accuracy:", acc)

In [ ]:
# GRU
model_gru = models.Sequential([
    layers.GRU(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    layers.Dense(len(le.classes_), activation="softmax")
])

model_gru.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_gru.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

loss, acc = model_gru.evaluate(X_test_rnn, y_test)
print("\n=== GRU ===")
print("Accuracy:", acc)



=== Logistic Regression ===
Accuracy: 0.86
              precision    recall  f1-score   support

           J       0.95      1.00      0.97        18
           K       0.88      0.88      0.88        17
           Q       0.83      0.59      0.69        17
           X       0.73      1.00      0.84        16
           Z       0.84      1.00      0.91        16
           Ñ       1.00      0.69      0.81        16

    accuracy                           0.86       100
   macro avg       0.87      0.86      0.85       100
weighted avg       0.87      0.86      0.85       100


=== Random Forest ===
Accuracy: 0.87
              precision    recall  f1-score   support

           J       1.00      1.00      1.00        18
           K       0.79      0.65      0.71        17
           Q       0.75      0.71      0.73        17
           X       1.00      0.94      0.97        16
           Z       0.80      1.00      0.89        16
           Ñ       0.88      0.94      0.91       

c:\Users\Angel\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.1562 - loss: 1.8050 - val_accuracy: 0.2625 - val_loss: 1.7824
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6469 - loss: 1.5364 - val_accuracy: 0.2875 - val_loss: 1.7413
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9219 - loss: 1.3256 - val_accuracy: 0.3875 - val_loss: 1.6966
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9781 - loss: 1.1251 - val_accuracy: 0.4500 - val_loss: 1.6457
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9969 - loss: 0.9385 - val_accuracy: 0.4875 - val_loss: 1.5992
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.7636 - val_accuracy: 0.5125 - val_loss: 1.5493
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.6085 - val_accuracy: 0.5500 - val_loss: 1.4966
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.4757 - val_accuracy: 0.5500 - val_loss: 1.4490
Ep

: 